<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/quantum_kernel_sweep_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane numpy scikit-learn matplotlib qiskit

In [ ]:
#!/usr/bin/env python3
"""
quantum_kernel_sweep.py

Reproduce per‐depth alignment & SVM cv‐accuracy for 2‐qubit
ZZFeatureMap kernels under different class‐imbalance ratios.
"""

import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from qiskit.circuit.library import ZZFeatureMap
from qiskit.quantum_info import Statevector, state_fidelity

def generate_data(n_samples: int,
                  imbalance: float,
                  random_state: int = 42):
    """
    Generate a 2D, binary toy dataset.
    imbalance = 1.0 --> balanced classes
    imbalance < 1.0 --> weight = [imbalance, 1-imbalance]
    """
    if imbalance >= 1.0:
        weights = [0.5, 0.5]
    else:
        weights = [imbalance, 1.0 - imbalance]

    X, y = make_classification(n_samples=n_samples,
                               n_features=2,
                               n_redundant=0,
                               n_clusters_per_class=1,
                               weights=weights,
                               class_sep=2.0,
                               random_state=random_state)
    return X, y

def build_feature_map(depth: int):
    """
    2-qubit ZZ feature map with CNOT entanglement.
    """
    return ZZFeatureMap(feature_dimension=2,
                        reps=depth,
                        entanglement='circular',
                        insert_barriers=False)

def compute_fidelity_kernel(feature_map, X):
    """
    Returns the (n×n) fidelity kernel K_ij = |<ψ(x_i)|ψ(x_j)>|^2
    by generating statevectors on the statevector simulator.
    """
    n = len(X)
    # precompute statevectors
    psi = []
    for x in X:
        qc = feature_map.assign_parameters(x)
        sv = Statevector.from_instruction(qc)
        psi.append(sv)

    K = np.zeros((n, n), dtype=float)
    for i in range(n):
        for j in range(i, n):
            fid = state_fidelity(psi[i], psi[j])**2
            K[i, j] = fid
            K[j, i] = fid
    return K

def center_kernel(K: np.ndarray):
    """
    Center the kernel matrix: Kc = H K H, H = I - (1/n) 11^T
    """
    n = K.shape[0]
    one_n = np.ones((n, n)) / n
    return K - one_n @ K - K @ one_n + one_n @ K @ one_n

def kernel_target_alignment(Kc: np.ndarray, y: np.ndarray):
    """
    Alignment between centered kernel Kc and label outer product y y^T.
    """
    yy = np.outer(y, y)
    num = np.trace(Kc @ yy)
    den = np.linalg.norm(Kc, 'fro') * np.linalg.norm(yy, 'fro')
    return num / den

def svm_cv_scores(K: np.ndarray, y: np.ndarray, max_splits: int = 5):
    """
    Runs StratifiedKFold CV on a precomputed kernel. Automatically
    reduces n_splits to avoid class‐count warnings.
    Returns sklearn scores array.
    """
    # find smallest class count
    classes, counts = np.unique(y, return_counts=True)
    min_count = counts.min()
    n_splits = min(max_splits, min_count)
    # if only one class present, return empty list
    if len(classes) < 2 or n_splits < 2:
        return np.array([])
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    svc = SVC(kernel='precomputed')
    # cross_val_score will slice K appropriately
    return cross_val_score(svc, K, y, cv=cv)

def main():
    np.random.seed(0)
    n_samples = 10
    imbalance_levels = [1.0, 0.75, 0.50]
    depths = [1, 3, 5]

    for imb in imbalance_levels:
        # generate data
        X, y = generate_data(n_samples, imb, random_state=1)
        # baseline: CNOT feature map at depth=3
        K_base = compute_fidelity_kernel(build_feature_map(depth=3), X)
        Kb_c = center_kernel(K_base)
        scores_base = svm_cv_scores(K_base, y)
        mean_b, std_b = scores_base.mean(), scores_base.std()

        # placeholder QNN pipeline – here we simply reuse baseline
        # In your real code you’d replace the below with:
        #   K_qnn = optimized_kernel( … )
        #   scores_qnn = svm_cv_scores(K_qnn, y)
        scores_qnn = scores_base.copy()
        mean_q, std_q = mean_b, std_b

        # print header
        header = f"Imbalance {int(imb*100)}%: "
        # print CNOT (no ± if std~=0)
        if std_b < 1e-6:
            header += f"CNOT={mean_b:.3f}, "
        else:
            header += f"CNOT={mean_b:.3f}±{std_b:.3f}, "
        # print QNN
        header += f"QNN={mean_q:.3f}±{std_q:.3f}"
        print(header)

        # per‐depth alignment & SVM
        for d in depths:
            fmap = build_feature_map(depth=d)
            K = compute_fidelity_kernel(fmap, X)
            Kc = center_kernel(K)
            align = kernel_target_alignment(Kc, y)
            scores = svm_cv_scores(K, y)
            if scores.size > 0:
                m, s = scores.mean(), scores.std()
                print(f"  depth={d} | align={align:.3f} | "
                      f"SVM={m:.3f}±{s:.3f}")
            else:
                print(f"  depth={d} | align={align:.3f} | SVM= n/a")
        print()

if __name__ == "__main__":
    main()